In [46]:
import tensorflow as tf
import numpy as np
import glob
tf.__version__

'2.0.0'

In [47]:
IMAGE_SIZE = 28

In [48]:
def color_preprocessing(x_train,x_test):
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train = x_train/255.
    x_test = x_test/255.
    return x_train,x_test

In [49]:
(x_train,y_train),(x_test,y_test) = tf.keras.datasets.mnist.load_data()
y_train = tf.keras.utils.to_categorical(y_train,10)
y_test = tf.keras.utils.to_categorical(y_test,10)
x_train,x_test = color_preprocessing(x_train,y_train)

In [50]:
x_train = np.reshape(x_train,(x_train.shape[0],x_train.shape[1],x_train.shape[2],1))

In [51]:
def representative_data_gen():
    
    dataset_list_index=np.random.choice(range(50000),100)
    for i in range(100):
        image = x_train[dataset_list_index[i]]
        image = tf.image.resize(image,[IMAGE_SIZE, IMAGE_SIZE])
        image = tf.cast(image,tf.float32)
        image = tf.expand_dims(image,0)
        #with tf.Session() as sess:     #tf2註解
            #image = sess.run(image)    #tf2註解
        yield [image]

In [52]:
keras_model = tf.keras.models.load_model("./mnist_lenet.h5")
converter = tf.lite.TFLiteConverter.from_keras_model(keras_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.uint8 
converter.inference_output_type = tf.uint8
converter.representative_dataset = representative_data_gen 
tflite_model = converter.convert()

In [53]:
with open('mnist115.tflite','wb') as f:
    f.write(tflite_model)